<a href="https://colab.research.google.com/github/fkutlu/bus_utilization_forcasting/blob/main/bus_utilization_forcasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bus Utilization Forescasting

In [ ]:
import pandas as pd
from pandas import datetime
from dateutil.parser import parse 
import matplotlib as mpl
import seaborn as sns
import numpy as np
import datetime
from datetime import datetime
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.figsize': (10, 7), 'figure.dpi': 120})

In [ ]:
# from google.colab import files
# uploaded = files.upload()
# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

In [ ]:
# Import Data
# df = pd.read_csv('municipality_bus_utilization.csv', index_col=0, infer_datetime_format=True)
df = pd.read_csv('https://raw.githubusercontent.com/fkutlu/bus_utilization_forcasting/main/municipality_bus_utilization.csv', index_col=0, infer_datetime_format=True)
df.reset_index(inplace=True)
print(df)

In [ ]:
# get rid of numbers for the seconds in order to group by hour for taking the max value
df['timestamp'] = df['timestamp'].apply(lambda x: (" ".join((x.split(':')))[:-6]))
# df['date'] = df['timestamp'].apply(lambda x: (x.split(' ')[0]))
# df['hour'] = df['timestamp'].apply(lambda x: (x.split(' ')[1]))
print(df)

### Portion data to train & test (last two weeks)

In [ ]:
# manual train test split
df_train = pd.DataFrame()
df_test = pd.DataFrame()

# apply seperately for each municipality
gb = df.groupby("municipality_id")

for group, rows in gb:
    # print(group)
    # print(rows)
    df2 = pd.DataFrame(data=rows)
    # print(df2.head(20))
    # take the maximum value for each hour
    gb2 = df2.groupby("timestamp").max('usage')
    grouped_df = gb2.reset_index()
    # create the hour of the day parameter for the analysis
    grouped_df['hour'] = grouped_df['timestamp'].apply(lambda x: (x.split(' ')[1]))
    curr_municipality_df = grouped_df.drop(columns = ['timestamp', 'total_capacity'])
    # print(curr_municipality_df.head(20))
    # spare latest two weeks for test
    num = len(curr_municipality_df.index)
    df_test = df_test.append(curr_municipality_df.iloc[num-102:num])
    # print(df_test.head(20))
    df_train = df_train.append(curr_municipality_df.iloc[0:(num-102)])

print(df_test)
print(df_train)
df_test.apply(pd.to_numeric).dtypes
df_test['hour'] = df_test['hour'].astype('int')
# print(df_test.dtypes)
# df_test = pd.to_numeric(df_test)
# df_test.describe()
df_train.apply(pd.to_numeric).dtypes
df_train['hour'] = df_train['hour'].astype('int')
print(df_train.dtypes)
# df_train.describe()

# 3D View in order to Determine Dependent and Target Values

In [ ]:
from mpl_toolkits import mplot3d
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(5, 10), dpi=120)

# Data for a three-dimensional line
# ax.plot3D(xline, yline, zline, 'gray')

# Data for three-dimensional scattered points
zdata = df_train.hour.values
xdata = df_train.municipality_id.values
ydata = df_train.usage.values

ax = plt.axes(projection='3d')
# ax = plt.figure().add_subplot(projection='3d')
ax.set_xlabel('MUNICIPALITY')
ax.set_ylabel('USAGE')
ax.set_zlabel('HOURS')

ax.view_init(elev=20., azim=-200)
ax.scatter(xdata, ydata, zdata, c=zdata, cmap='viridis', linewidth=1);

# Train Model, Test and Evaluate

In [ ]:
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler

y_train = df_train.usage
X_train_df = df_train.drop('usage', axis=1)

y_test = df_test.usage.to_numpy()
print(y_test)
X_test_df = df_test.drop('usage', axis=1)

scaler = StandardScaler()
scaler.fit(X_train_df)  # No cheating, never scale on the training+test!
X_train = scaler.transform(X_train_df)
X_test = scaler.transform(X_test_df)

X_train_df = pd.DataFrame(X_train, columns=X_train_df.columns)
X_test_df = pd.DataFrame(X_test, columns=X_test_df.columns)

# Bayesian regression
reg = linear_model.BayesianRidge()
reg.fit(X_train, y_train)
yhat = reg.predict(X_test)
print(yhat)

forecast_errors = [y_test[i]-yhat[i] for i in range(len(y_test))]
print('Forecast Errors: %s' % forecast_errors)

bias = sum(forecast_errors) * 1.0/len(y_test)
print('Bias: %f' % bias)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, yhat)
print('Mean Absolute Error (MAE): %f' % mae)

from sklearn.metrics import mean_squared_error
from math import sqrt
mse = mean_squared_error(y_test, yhat)
rmse = sqrt(mse)
print('Root Mean Squared Error (RMSE): %f' % rmse)

In [ ]:
plt.plot(y_test)
plt.plot(yhat, color='red')
plt.title('Bus usage prediction (RED plots) by Bayesian Regression', fontsize=16)
plt.tight_layout()

In [ ]:
# Using scipy: Subtract the line of best fit
from scipy import signal
detrended = signal.detrend(df_train.usage.values)
plt.plot(detrended)
plt.title('Bus Usage detrended by subtracting the least squares fit', fontsize=12)